# Clustering Crypto

In [206]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [207]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE

crypto_df = pd.read_csv('crypto_data.csv', index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [208]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE

crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [209]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE

crypto_df = crypto_df.drop(['IsTrading'], axis=1)
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [210]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE

crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [211]:
# Keep the rows where coins are mined.
# YOUR CODE HERE

mined_df = crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] < 1].index)
mined_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [212]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE

coinname_df = mined_df['CoinName']
coinname_df.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [213]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
mined_df=mined_df.drop(['CoinName'], axis=1)
mined_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [214]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE

x = pd.get_dummies(data=mined_df, columns=['Algorithm', 'ProofType'])
x.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE

x_scaled = StandardScaler().fit_transform(x)
x_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [216]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE

pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(x_scaled)
crypto_pca

array([[-0.33743454,  1.07343795, -0.52041726],
       [-0.32077838,  1.0738074 , -0.52061405],
       [ 2.31054673,  1.69165417, -0.53363433],
       ...,
       [ 0.32422273, -2.30634306,  0.34602552],
       [-0.13880296, -2.09875842,  0.41090289],
       [-0.29579006,  0.86490507, -0.28290885]])

In [257]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE

pcs_df = pd.DataFrame(
    data=crypto_pca, columns=['PC1', 'PC2', 'PC3'], index=mined_df.index)

pcs_df.head()

,PC1,PC2,PC3
42,-0.337435,1.073438,-0.520417
404,-0.320778,1.073807,-0.520614
1337,2.310547,1.691654,-0.533634
BTC,-0.147462,-1.348867,0.167158
ETH,-0.156983,-2.047551,0.322434


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [219]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE

inertia=[]
k = (range(1,10))

for i in k:
    km=KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df_pca)
    inertia.append(km.inertia_)
    
elbow_data = {'k':k, 'inertia':inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks=k, title='Elbow Curve')


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [243]:
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = data.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["Class"] = model.labels_

    return data

In [258]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4, random_state=42)

# Fit the model
# YOUR CODE HERE
model.fit(pcs_df)

# Predict clusters
# YOUR CODE HERE
predictions = model.predict(pcs_df)
print(predictions)
    

[1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1
 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0
 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0
 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1
 1 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1
 0 0 1 0 0 0 0 1 3 3 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0
 1 1 1 1 0 1 1 0 1 1 0 3 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 0 0 0 1 1 1 0 1 0 1 0 1 

In [259]:
pcs_df['Class'] = model.labels_
pcs_df.head()


,PC1,PC2,PC3,Class
42,-0.337435,1.073438,-0.520417,1
404,-0.320778,1.073807,-0.520614,1
1337,2.310547,1.691654,-0.533634,1
BTC,-0.147462,-1.348867,0.167158,0
ETH,-0.156983,-2.047551,0.322434,0


In [247]:
pcs_df.dtypes

PC 1     float64
PC 2     float64
PC 3     float64
Class      int32
dtype: object

In [260]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE

clustered_df = mined_df.merge(pcs_df, left_index=True, right_index=True)


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

clustered_df['Coinname'] = coinname_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class,Coinname
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337435,1.073438,-0.520417,1,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320778,1.073807,-0.520614,1,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310547,1.691654,-0.533634,1,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.147462,-1.348867,0.167158,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.156983,-2.047551,0.322434,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.169953,-1.092766,-0.028744,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387542,1.233850,-0.445730,1,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.148225,-2.248499,0.401865,0,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.155426,-2.047629,0.322428,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.138802,-2.098758,0.410903,0,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [272]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE

fig = px.scatter_3d(clustered_df, 
                    x="PC1", 
                    y="PC2", 
                    z="PC3", 
                    hover_name="Coinname", 
                    hover_data=["Algorithm"],
                    color="Class", 
                    symbol="Class", 
                    width=800,)

fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [273]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

clustered_df.hvplot.table(columns=['Coinname', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)


:Table   [Coinname,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [286]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

tradable_count = len(clustered_df.index)
print(f'There are {tradable_count} tradable cryptocurrencies')

There are 532 tradable cryptocurrencies


In [20]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_scaled = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
clustered_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [293]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
clustered_scaled_df = pd.DataFrame(clustered_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df = clustered_scaled_df.merge(clustered_df[['Coinname', 'Class']], left_index=True, right_index=True)

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,Coinname,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [295]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', hover_cols='Coinname')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,Coinname)